In [7]:
%pylab inline
import sys
sys.path.append('/vagrant_data/ecool_project/rswarp/') # need rswarp utils in path, kludge for now
from rswarp.utilities.file_utils import readparticles, loadparticlefiles
from rswarp.utilities.beam_analysis import convertunits, plotphasespace
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np

#Import phase space data from all steps. Uses readparticles()
stepArrays = loadparticlefiles('diags/hdf5/')

fig = plt.figure(figsize=(12, 8))
ax0 = plt.subplot(2,3,1)
ax1 = plt.subplot(2,3,2)
ax2 = plt.subplot(2,3,3)
ax3 = plt.subplot(2,3,4)
ax4 = plt.subplot(2,3,5)
sc0 = ax0.axes.plot([],[],linestyle='',marker='o',markersize=2.0,color='b')[0]
sc1 = ax1.axes.plot([],[],linestyle='',marker='o',markersize=2.0,color='b')[0]
sc2 = ax2.axes.plot([],[],linestyle='',marker='o',markersize=2.0,color='b')[0]
sc3 = ax3.axes.plot([],[],linestyle='',marker='o',markersize=2.0,color='b')[0]
sc4 = ax4.axes.plot([],[],linestyle='',marker='o',markersize=2.0,color='b')[0]

def adjustaxis(ax,mar=0.2,equalize=True):
    ax.set_xmargin(mar)
    ax.set_ymargin(mar)
    ax.relim(True)
    ax.autoscale()
    if equalize:
        xsc = max(np.abs(ax.get_xlim()))
        ysc = max(np.abs(ax.get_ylim()))
        ax.set_xlim(-xsc,xsc)
        ax.set_ylim(-ysc,ysc)

def plotphasespace(particlearray):
    sc0.set_data(particlearray[:, 0], particlearray[:, 1])
    ax0.set_title("x-xp phase space")
    ax0.set_xlabel("x (m)")
    ax0.set_ylabel("xp (rad)")

    sc1.set_data(particlearray[:, 2], particlearray[:, 3])
    ax1.set_title("y-yp phase space")
    ax1.set_xlabel("y (m)")
    ax1.set_ylabel("yp (rad)")
    

    sc2.set_data(particlearray[:, 0], particlearray[:, 2])
    ax2.set_title("x-y distribution")
    ax2.set_xlabel("x (m)")
    ax2.set_ylabel("y (m)")
    

    sc3.set_data(particlearray[:, 4], particlearray[:, 5])
    ax3.set_title("z-pz phase space")
    ax3.set_xlabel("z (m)")
    ax3.set_ylabel("pz (MeV/c)")
    

    sc4.set_data(particlearray[:, 4], particlearray[:, 0])
    ax4.set_title("z-x phase space")
    ax4.set_xlabel("z (m)")
    ax4.set_ylabel("x (m)")
    
    
    return sc0,sc1,sc2,sc3,sc4

def animate(i):
    #Convert units of phase space
    a = convertunits(stepArrays[sort(stepArrays.keys())[i-1]]['p'])
    #Plot phase space of one of the snapshots
#     adjustaxis(ax0)
#     adjustaxis(ax1)
#     adjustaxis(ax2)
#     adjustaxis(ax3,equalize=False)
#     adjustaxis(ax4,mar=0.05)
    return plotphasespace(a)

def init():
    a = convertunits(stepArrays[sort(stepArrays.keys())[0]]['p'])
    scs = plotphasespace(a)
#     adjustaxis(ax0)
#     adjustaxis(ax1)
#     adjustaxis(ax2)
    adjustaxis(ax3,equalize=False)
#     adjustaxis(ax4,mar=0.05)
    ax0.set_xlim(-0.01,0.01)
    ax0.set_ylim(-0.02,0.02)
    ax1.set_xlim(-2e16,2e16)
    ax1.set_ylim(-0.015,0.015)
    ax2.set_xlim(-0.02,0.02)
    ax2.set_ylim(-2.5e16,2.5e16)
    ax3.set_xlim(-0.2,6)
#     ax3.set_ylim(0.015,0.025)
    ax4.set_xlim(-0.2,6)
    ax4.set_ylim(-0.02,0.02)
    return scs

# animation code skeleton from http://jakevdp.github.io/blog/2013/05/12/embedding-matplotlib-animations/
ani = animation.FuncAnimation(fig, animate, np.arange(1, len(stepArrays)), init_func=init,
                              interval=25, blit=False)

from tempfile import NamedTemporaryFile

VIDEO_TAG = """<video controls>
 <source src="data:video/x-m4v;base64,{0}" type="video/mp4">
 Your browser does not support the video tag.
</video>"""

def anim_to_html(anim):
    if not hasattr(anim, '_encoded_video'):
        with NamedTemporaryFile(suffix='.mp4') as f:
            anim.save(f.name, fps=20, extra_args=['-vcodec', 'libx264'])
            video = open(f.name, "rb").read()
        anim._encoded_video = video.encode("base64")
    
    return VIDEO_TAG.format(anim._encoded_video)

from IPython.display import HTML

def display_animation(anim):
    plt.close(anim._fig)
    return HTML(anim_to_html(anim))


# animate(100)
display_animation(ani)

Populating the interactive namespace from numpy and matplotlib
